In [6]:
import pandas as pd
import numpy as np
import os
import psycopg2

In [7]:
df = pd.DataFrame(pd.date_range('1/1/2021','12/31/2023'), columns=['date'])
df

,date
0,2021-01-01
1,2021-01-02
2,2021-01-03
3,2021-01-04
4,2021-01-05
...,...
1090,2023-12-27
1091,2023-12-28
1092,2023-12-29
1093,2023-12-30


In [9]:
df['year'] = df['date'].dt.year
df['quarter_number'] = df['date'].dt.quarter
df['quarter_text'] = df['date'].apply(lambda x: f'Q{x.quarter} {x.strftime("%Y")}')
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.strftime("%B")
df['year_month'] = df['date'].dt.strftime("%B %Y")
df['week'] = df['date'].dt.isocalendar().week
df['year_week'] = df['date'].apply(lambda x: f'{x.isocalendar()[0]}, Week {x.isocalendar()[1]}')
df['weekday'] = df['date'].dt.strftime("%A")
df

,date,year,quarter_number,quarter_text,month,year_month,week,year_week,weekday,month_name
0,2021-01-01,2021,1,Q1 2021,1,January 2021,53,"2020, Week 53",Friday,January
1,2021-01-02,2021,1,Q1 2021,1,January 2021,53,"2020, Week 53",Saturday,January
2,2021-01-03,2021,1,Q1 2021,1,January 2021,53,"2020, Week 53",Sunday,January
3,2021-01-04,2021,1,Q1 2021,1,January 2021,1,"2021, Week 1",Monday,January
4,2021-01-05,2021,1,Q1 2021,1,January 2021,1,"2021, Week 1",Tuesday,January
...,...,...,...,...,...,...,...,...,...,...
1090,2023-12-27,2023,4,Q4 2023,12,December 2023,52,"2023, Week 52",Wednesday,December
1091,2023-12-28,2023,4,Q4 2023,12,December 2023,52,"2023, Week 52",Thursday,December
1092,2023-12-29,2023,4,Q4 2023,12,December 2023,52,"2023, Week 52",Friday,December
1093,2023-12-30,2023,4,Q4 2023,12,December 2023,52,"2023, Week 52",Saturday,December


In [17]:
df.columns = [x.lower().replace(" ","_").replace("%","")for x in df.columns]
df.columns

Index(['date', 'year', 'quarter_number', 'quarter_text', 'month', 'year_month',
       'week', 'year_week', 'weekday', 'month_name'],
      dtype='object')

In [12]:
replacements = {
    'object' : 'varchar',
    'int64' : 'varchar',
    'datetime64[ns]': 'date',
    'datetime64': 'timestamp',
    'UInt32' : 'varchar'
}
replacements

{'object': 'varchar',
 'int64': 'varchar',
 'datetime64[ns]': 'date',
 'datetime64': 'timestamp',
 'UInt32': 'varchar'}

In [13]:
col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(df.columns, df.dtypes.replace(replacements)))
col_str

'date date, year varchar, quarter_number varchar, quarter_text varchar, month varchar, year_month varchar, week varchar, year_week varchar, weekday varchar, month_name varchar'

In [14]:
conn_string = "host= db.app.production.jetstreamafrica.com \
                dbname='jetstream-finance' \
                    user='ujetstreamfinance' password='Smehr5A7FGw4jEX@' "

conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

print('opened database successfully')

opened database successfully


In [15]:
cursor.execute("drop table if exists dimdate;")

In [16]:
cursor.execute("create table dimdate \
    (date date, year varchar, quarter_number varchar, quarter_text varchar, month varchar, year_month varchar, week varchar, year_week varchar, weekday varchar, month_name varchar)")

In [18]:
#insert values to tables
#save df to csv
df.to_csv('dimdate', header=df.columns, index=False, encoding='utf-8')
#open the csv file, save it as an object and upload to db
my_file = open("dimdate")
print('file opened in memory')

file opened in memory


In [19]:
SQL_STATEMENT = """
COPY dimdate FROM STDIN WITH
   CSV
   HEADER
   DELIMITER AS ','
"""
cursor.copy_expert(sql = SQL_STATEMENT, file=my_file)
print('file copied to db')

file copied to db


In [20]:
cursor.execute("grant select on table dimdate to public")
conn.commit()

cursor.close()
print('table dimdate imported successfully')

table dimdate imported successfully
